### Imports

In [1]:
import pandas as pd
import numpy as np
import json
import time 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
import warnings
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import re
import requests as req
from bs4 import BeautifulSoup as bs
import random
import http.client, urllib.parse
import csv
import seaborn as sns
#from sample_building_functions import *


warnings.simplefilter('ignore')

### Defining some useful functions

In [9]:
def outliers_border_tukey(data):
    """
    Detecta outliers utilizando el método de Tukey.

    Parámetros:
    data (list or np.array): un arreglo unidimensional de valores numéricos.
    
    - Written by chatGPT, modified by DiSanchz
    """
    # Calculamos el rango intercuartil (IQR)
    q1, q3 = np.percentile(data, [25, 75])
    iqr = q3 - q1

    # Establecemos los límites inferior y superior
    lower_bound = q1 - (1.5 * iqr)
    upper_bound = q3 + (1.5 * iqr)
    
    print(f'The upper and lower bounds for outliers are: {(lower_bound, upper_bound)}')

    return (lower_bound, upper_bound)

## 1. Loading data on all cities _vanilla_ sample DataFrame "all_cities_filter"

In [ ]:
all_cities_filter = pd.read_csv("data/all_cities_filtered.csv")

## 2. complete data with GDP and HDI figures

### Loading reference indicators' data

In [15]:
GDP_raw = pd.read_csv('../data/ignore/GDP_per_capita_2019.csv', encoding= 'unicode_escape', sep=';')
HDI_raw = pd.read_csv('../data/ignore/HDI_index_historical.csv', encoding= 'unicode_escape')

### 2.1 GDP

In [16]:
GDP_raw['country'] =  [i.lower() for i in GDP_raw['GDP per capita, current prices\n (U.S. dollars per capita)']]
GDP_raw.drop(['GDP per capita, current prices\n (U.S. dollars per capita)'], inplace=True, axis=1)

"Cleaning" country names. A dictionary of alternative country names is created to map each contry name as referred in "all_cities_filter" compared to GDP data source for those cases where it diverges.

In [17]:
fixed_names = {}
fixed_names ['burkinafaso'] = 'burkina faso'
fixed_names ['centralafrica'] = 'central african republic'
fixed_names ['drcongo'] = 'congo, dem. rep. of the'
fixed_names ['equatorialguinea'] = 'equatorial guinea'
fixed_names ['guineabissau'] = 'guinea-bissau'
fixed_names ['ivorycoast'] = "cote divoire"
fixed_names ['sierraleone'] = "sierra leone"
fixed_names ['ivorycoast'] = 'south sudan'
fixed_names['costarica'] = 'costa rica'
fixed_names['southsudan'] = 'south sudan'
fixed_names['turkey'] = 'türkiye, republic of'
fixed_names['domrep'] = 'dominican republic'
fixed_names['elsalvador'] = 'el salvador'
fixed_names['puertorico'] = 'puerto rico'  
fixed_names['usa'] = 'united states'  
fixed_names['Azerbaijan.html'] = 'azerbaijan'
fixed_names['kyrgyzstan'] = 'kyrgyz republic'
fixed_names['laos'] = 'lao p.d.r.'
fixed_names['saudiarabia'] = 'saudi arabia'
fixed_names['srilanka'] = 'sri lanka'
fixed_names['taiwan'] = 'taiwan province of china'
fixed_names['bosnia'] = 'bosnia and herzegovina'
fixed_names['czechrep'] = 'czech republic'
fixed_names['uk'] = 'united kingdom'
fixed_names['northmacedonia'] = 'north macedonia '
fixed_names['russia'] = 'russian federation'
fixed_names['slovakia'] = 'slovak republic'
fixed_names['newzealand'] = 'new zealand'
fixed_names['papuanewguinea'] = 'papua new guinea'

Retrieving GDP data and inserting it into the cities DF

In [18]:
GDP_column = []

for i in all_cities_filter['country']:
    try:
        try:
            GDP_column.append(GDP_raw[GDP_raw.eq(i).any(1)].iloc[0][0])
        except:
            GDP_column.append(GDP_raw[GDP_raw.eq(fixed_names.get(i)).any(1)].iloc[0][0])
    except:
        GDP_column.append(np.nan)
        pass
        
all_cities_GDP = all_cities_filter.copy()
all_cities_GDP['GDP_19'] = GDP_column
all_cities_GDP_valid = all_cities_GDP[all_cities_GDP.GDP_19.notnull()]

In [19]:
all_cities_GDP_valid

,cityname,country,latest_figure,figure_description,town_code,count,GDP_19
0,Adrar,algeria,68276,(C)2008-04-14Area,0,1,"3953,576"
1,Aflou,algeria,93585,(C)2008-04-14Area,1,1,"3953,576"
2,Aïn Béïda,algeria,116064,(C)2008-04-14Area,2,1,"3953,576"
3,Aïn Defla,algeria,55259,(C)2008-04-14Area,3,1,"3953,576"
4,Aïn M'lila,algeria,65371,(C)2008-04-14Area,4,1,"3953,576"
...,...,...,...,...,...,...,...
5471,Rockhampton,australia,63151,(C)2021-08-11Area,5471,1,"54323,089"
5472,Sunshine Coast,australia,284131,(C)2021-08-11Area,5472,1,"54323,089"
5473,Toowoomba,australia,108398,(C)2021-08-11Area,5473,1,"54323,089"
5474,Townsville,australia,173724,(C)2021-08-11Area,5474,1,"54323,089"


### 2.2 HDI

In [20]:
HDI_raw['country_format']= [i.lower().replace(" ", "") for i in HDI_raw['country']]
HDI_short=HDI_raw[['country', 'country_format', 'hdi_2019']]

"Cleaning" country names. A dictionary of alternative country names is created to map each contry name as referred in "all_cities_filter" compared to GDP data source for those cases where it diverges.

In [22]:
fixed_names = {}
fixed_names ['burkinafaso'] = 'burkina faso'
fixed_names ['centralafrica'] = 'centralafricanrepublic'
fixed_names ['drcongo'] = 'congo(democraticrepublicofthe)'
fixed_names ['equatorialguinea'] = 'equatorial guinea'
fixed_names ['guineabissau'] = 'guinea-bissau'
fixed_names ['ivorycoast'] = "cote"
fixed_names ['sierraleone'] = "sierra leone"
fixed_names ['ivorycoast'] = 'cote'
fixed_names['costarica'] = 'costa rica'
fixed_names['southsudan'] = 'south sudan'
fixed_names['turkey'] = 'türkiye, republic of'
fixed_names['domrep'] = 'dominican republic'
fixed_names['elsalvador'] = 'el salvador'
fixed_names['puertorico'] = 'puerto rico'  
fixed_names['usa'] = 'unitedstates'  
fixed_names['Azerbaijan.html'] = 'azerbaijan'
fixed_names['kyrgyzstan'] = 'kyrgyz republic'
fixed_names['laos'] = 'lao p.d.r.'
fixed_names['saudiarabia'] = 'saudi arabia'
fixed_names['srilanka'] = 'sri lanka'
fixed_names['taiwan'] = 'taiwan province of china'
fixed_names['bosnia'] = 'bosniaandherzegovina'
fixed_names['czechrep'] = 'czechia'
fixed_names['uk'] = 'unitedkingdom'
fixed_names['northmacedonia'] = 'north macedonia '
fixed_names['russia'] = 'russianfederation'
fixed_names['slovakia'] = 'slovak republic'
fixed_names['newzealand'] = 'new zealand'
fixed_names['papuanewguinea'] = 'papua new guinea'

In [23]:
HDI_column = []

for i in all_cities_GDP_valid['country']:
    try:
        try:
            HDI_column.append(HDI_short[HDI_short.eq(i).any(1)].iloc[0][2])
        except:
            try:
                #print(f"{i} not found in first attempt, moving to look in alternative dictionary")
                #print(f"{i} found as {fixed_names.get(i)} in alternative source, proceeding with second attempt")
                #HDI_column.append(HDI_short[HDI_short.eq(fixed_names.get(i)).any(1)].iloc[0][2])
                HDI_column.append(HDI_short[HDI_short['country_format']== fixed_names.get(i)].iloc[0][2])
            except:
                HDI_column.append(np.nan)
                #print(f"{i} not found in second attempt")
    except:
        
        pass
        
all_cities_GDP_HDI = all_cities_GDP_valid.copy()
all_cities_GDP_HDI['HDI_19'] = HDI_column
all_cities_GDP_HDI_valid = all_cities_GDP_HDI[all_cities_GDP_HDI.HDI_19.notnull()]

### 3. Population figures adjustment [IN PROGRESS - NOT CLEAN NOT FINISHED]

Extracting the year in which the population figure for a given city was made/estimated

In [66]:
# Creating new column with year of estimate for latest_figure
all_cities_GDP_HDI_valid['year_latest_figure'] = [int(re.findall(r"\)(\d{4})", i)[0]) for i in all_cities_GDP_HDI_valid['figure_description']]

In [93]:
pop_raw = pd.read_csv('data/tp/WPP2022.csv', sep=',', encoding='utf8', quotechar='"')

In [105]:
pop_raw['location_fixed'] = [i.lower().replace(" ", "") for i in pop_raw['Location']]

In [215]:
fixed_names = {}
fixed_names ['burkinafaso'] = 'burkina faso'
fixed_names ['centralafrica'] = 'centralafricanrepublic'
fixed_names ['drcongo'] = 'democraticrepublicofthecongo'
fixed_names ['equatorialguinea'] = 'equatorial guinea'
fixed_names ['guineabissau'] = 'guinea-bissau'
fixed_names ['ivorycoast'] = "côted'ivoire"
fixed_names ['laos'] = "laopeople'sdemocraticrepublic"
fixed_names['tanzania'] = 'unitedrepublicoftanzania'
fixed_names['bolivia'] = 'bolivia(plurinationalstateof)'
fixed_names['turkey'] = 'türkiye'
fixed_names['domrep'] = 'dominicanrepublic'
fixed_names['elsalvador'] = 'el salvador'
fixed_names['puertorico'] = 'puerto rico'  
fixed_names['usa'] = 'unitedstatesofamerica'  
fixed_names['Azerbaijan.html'] = 'azerbaijan'
fixed_names['kyrgyzstan'] = 'kyrgyz republic'
fixed_names['saudiarabia'] = 'saudi arabia'
fixed_names['srilanka'] = 'sri lanka'
fixed_names['taiwan'] = 'taiwan province of china'
fixed_names['bosnia'] = 'bosniaandherzegovina'
fixed_names['czechrep'] = 'czechia'
fixed_names['uk'] = 'unitedkingdom'
fixed_names['iran'] = 'iran(islamicrepublicof)'
fixed_names['russia'] = 'russianfederation'
fixed_names['newzealand'] = 'new zealand'
fixed_names['venezuela'] = 'venezuela(bolivarianrepublicof)'

In [255]:
# Computing rates of popn. change for each country
# for the year of last estimate compared to 2019

pop_adjust = {}

for i in list(all_cities_GDP_HDI_valid['country'].unique()):
    
    print(i)
    try:
        a = pop_2019(i)
        b = lookfor_pop(all_cities_GDP_HDI_valid[all_cities_GDP_HDI_valid['country'] == i ].iloc[0][-1], i)
        c = a/b
        pop_adjust[i] = c
    except:
        a = pop_2019(fixed_names.get(i))
        b = lookfor_pop(all_cities_GDP_HDI_valid[all_cities_GDP_HDI_valid['country'] == i ].iloc[0][-1], fixed_names.get(i))
        c = a/b
        pop_adjust[i] = c     

algeria
benin
botswana
burkinafaso
burundi
cameroon
centralafrica
chad
drcongo
djibouti
egypt
equatorialguinea
eritrea
ethiopia
gabon
ghana
guinea
guineabissau
ivorycoast
kenya
madagascar
malawi
mali
mauritania
mauritius
morocco
mozambique
namibia
niger
nigeria
rwanda
senegal
sierraleone
southsudan
sudan
tanzania
togo
tunisia
uganda
zambia
zimbabwe
argentina
bolivia
brazil
canada
chile
colombia
costarica
domrep
ecuador
elsalvador
guatemala
guyana
haiti
honduras
jamaica
mexico
nicaragua
panama
paraguay
peru
suriname
usa
uruguay
venezuela
afghanistan
Azerbaijan.html
bangladesh
cambodia
georgia
india
indonesia
iran
japan
jordan
kazakhstan
kyrgyzstan
laos
lebanon
malaysia
mongolia
nepal
oman
pakistan
philippines
qatar
saudiarabia
srilanka
tajikistan
thailand
turkmenistan
uzbekistan
vietnam
albania
austria
belarus
belgium
bosnia
bulgaria
croatia
czechrep
denmark
estonia
finland
france
germany
uk
greece
hungary
iceland
ireland
italy
latvia
lithuania
netherlands
northmacedonia
norway
poland
p

In [198]:
def lookfor_pop(year, country):
    ''' 
    working
    '''
    tmp = pop_raw[pop_raw['location_fixed'] == country]
    return int(str(tmp[tmp['Time'] == year].iloc[0][-3]).replace(".",""))
    

In [197]:
def latest_figure_pop(country):
    '''
    working
    '''
    tmp = pop_raw[pop_raw['location_fixed'] == country]
    return int(str(tmp[tmp['Time'] == all_cities_GDP_HDI_valid[all_cities_GDP_HDI_valid['country'] == country].iloc[0][8]].iloc[0][-3]).replace(".",""))

In [213]:
def latest_figure_pop_robust(country):
    '''
    working
    '''
    tmp = pop_raw[pop_raw['location_fixed'] == fixed_names.get(country)]
    return int(str(tmp[tmp['Time'] == all_cities_GDP_HDI_valid[all_cities_GDP_HDI_valid['country'] == country].iloc[0][8]].iloc[0][-3]).replace(".",""))

In [196]:
def pop_2019(country):
    '''
    working
    '''
    return lookfor_pop(2019, country)

In [220]:
a = pop_2019('spain')
print(f'pop 2019: {a}')
b = latest_figure_pop('spain')
print(f'pop in year of latest est.: {b}')
a/b

pop 2019: 47131372
pop in year of latest est.: 47486935


0.9925124036748213

In [221]:
47486935*0.9925124036748213

47131372.0

0.007544083376142752

In [264]:
32*20.5

656.0

In [258]:
to_correct = []

for i in pop_adjust.keys():
    
    if pop_adjust.get(i) > 1.1:
        print(i)
        print(pop_adjust.get(i))
        
    elif pop_adjust.get(i) < 0.9:
        print(i)
        print(pop_adjust.get(i))

algeria
1.2353448660892499
benin
11.92236483058534
burundi
1.434486789193039
cameroon
1.4924506970148081
chad
1.4028084934336151
drcongo
0.16401695597387073
djibouti
1.19186596870724
equatorialguinea
21.591766652300247
eritrea
1.5268273460754482
gabon
1.1790318290255732
ghana
0.12325566509645718
guinea
1.13625026635955
guineabissau
1.2902449980225197
ivorycoast
1.1370697945755168
mali
1.3681848857502361
mauritania
1.1712254929856298
namibia
11.473986324882523
niger
1.3057180334610885
nigeria
20.812176154134047
rwanda
1.1840066920447285
senegal
1.1769117225424819
southsudan
1.1840206947322993
sudan
1.3482541110481596
tanzania
1.2529277894967739
togo
9.76371440957622
uganda
0.0967221174395605
zambia
1.3326828878532224
bolivia
1.1142528494430823
domrep
1.1131500329130588
ecuador
0.11570527136008102
haiti
1.1469379500090333
nicaragua
9.72758935791923
panama
1.168040662133995
afghanistan
9.691387688105094
Azerbaijan.html
10.37438409161878
bangladesh
1.101891449298272
india
0.109978430699010

A veces se come un dígito en las cifras el cacharro este...

Ha pasado en:

- Australia
- Nigeria

También es posible que simplemente algo falle en el cómputo,

es el caso de:

- Turquía


In [230]:
look = pop_raw[pop_raw['location_fixed'] == 'nigeria']
look[look['Time'] == 1990]

,SortOrder,LocID,Notes,ISO3_code,ISO2_code,SDMX_code,LocTypeID,LocTypeName,ParentID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal,PopDensity,location_fixed
293221,86.0,566,NaN,NGA,NG,566.0,4.0,Country/Area,914.0,Nigeria,2,Medium,1990,1990.5,47821.647,47392.609,95214.257,104.543,nigeria


In [251]:
lookfor_pop(2019, 'russianfederation')

145742286

In [252]:
lookfor_pop(2021, 'russianfederation')

145102755

In [253]:
all_cities_GDP_HDI_valid[all_cities_GDP_HDI_valid['country'] == 'turkey' ].iloc[0][-1]

2021

In [96]:
all_cities_GDP_HDI_valid

,cityname,country,latest_figure,figure_description,town_code,count,GDP_19,HDI_19,year_latest_figure
0,Adrar,algeria,68276,(C)2008-04-14Area,0,1,"3953,576",0.748,2008
1,Aflou,algeria,93585,(C)2008-04-14Area,1,1,"3953,576",0.748,2008
2,Aïn Béïda,algeria,116064,(C)2008-04-14Area,2,1,"3953,576",0.748,2008
3,Aïn Defla,algeria,55259,(C)2008-04-14Area,3,1,"3953,576",0.748,2008
4,Aïn M'lila,algeria,65371,(C)2008-04-14Area,4,1,"3953,576",0.748,2008
...,...,...,...,...,...,...,...,...,...
5488,Tauranga,newzealand,158300,(Ep)2022-06-30Area,5488,1,"42342,994",0.937,2022
5489,Wellington,newzealand,212000,(Ep)2022-06-30Area,5489,1,"42342,994",0.937,2022
5490,Whangārei,newzealand,54900,(Ep)2022-06-30Area,5490,1,"42342,994",0.937,2022
5491,Lae,papuanewguinea,148934,(Cf)2011-07-10Area,5491,1,"2877,577",0.560,2011
